In [ ]:
!pip install --quiet transformers datasets accelerate nvidia-ml-py3
!pip install -U adapter-transformers

In [1]:
from transformers import AutoTokenizer 
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2-xl")
tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
train_path = './YourDatasetHere.txt'

/home/skyler/Desktop/yes/envs/transformers/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(train_path, 'r') as f:
    for i in range(5):
        line = f.readline()
        print(line)



 Hello! <

 Greetings! My name is Methode, what would you like to do today? 😊 >

 I would like to talk about the Universe! About the stars and planets! <

 Wonderful! What is it about the stars that fascinates you the most? 😊 >



In [3]:
# Add a new adapter
model.add_adapter("methode")
# Add a matching classification head

# Activate the adapter
model.train_adapter("methode")

In [4]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=200
          )
          
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, data_collator

train_dataset, data_collator = load_dataset(train_path, tokenizer)

/home/skyler/Desktop/yes/envs/transformers/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [15]:
training_args = TrainingArguments(
    output_dir="./bomba",
    overwrite_output_dir=True,
    num_train_epochs=40,
    per_device_train_batch_size=1,
    save_total_limit=10,              # limit on the total number of checkpoints to save
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,    #set the gradient_checkpointing=True, to recude vram usage
    save_steps=4000,
    #warmup_steps=500,
    prediction_loss_only=True,
    optim="adafactor",
    #optim="adafactor"
    #optim="sgd"
    logging_steps=50,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
from transformers import AdapterTrainer

trainer = AdapterTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        data_collator=data_collator, 
    )

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()
model.save_adapter("methode_adapter", "methode")

In [ ]:
model.half()

In [ ]:
#model.to('cuda')
prompt = tokenizer("Hello! Who are you? < ", return_tensors='pt')
prompt = {key: value.to('cuda') for key, value in prompt.items()}
out = model.generate(**prompt, min_length=90, max_length=200, do_sample=True, temperature=0.4 ,no_repeat_ngram_size=3)
tokenizer.decode(out[0])

In [ ]:
generated_text = text_generator("I'm an AI with the purpose of", max_length=100, num_return_sequences=1, temperature=0.8, do_sample=True, no_repeat_ngram_size=6, bad_words_ids=[[tokenizer("<<")["input_ids"][0]]])[0]["generated_text"]

print(generated_text)

The End